In [0]:
import os
import warnings
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd


In [44]:
# 구글 드라이브 연동
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [45]:
faqs = pd.read_csv(os.path.join('data','/content/drive/My Drive/data/kor_elec_faq.csv'), encoding='CP949')
faqs

,순번,조회수,업무분류,업무분류명,등록일,수정일,제목,내용,FAQ분류
0,554,"10,912",20023,전기사용:할인요금,20030214,20180807,1주택수가구 요금을 적용받고 있는데 가구수가 감소하면 어떻게 해야 합니까.,1주택수가구 요금을 적용받던 고객이 세입자 전출등 가구수가 감소하는 경우에는 1개월...,전기요금:전기요금 계산
1,229,"3,875",20100,전기사용:신규업무:공급방법/장소,20030214,20190211,고객이 임시공급설비 설치공사비를 부담하여 한전에서 임시공급설비를 시설하였을 경우의 ...,임시공급설비를 한전에서 설치할 경우에는 상시전력 공급설비의 수급지점과 동일하게하여 ...,전기사용:전기사용신청
2,535,"20,176",20100,전기사용:신규업무:공급방법/장소,20030214,20180807,"변압기공동이용(모자거래)이란 무엇이며, 요금계산은 어떻게 합니까.",동일 전기사용장소내에 전기사용용도(계약종별)가 다르거나 회계단위가 상이한 2이상의 ...,전기사용:전기사용계약
3,117,"4,461",20100,전기사용:신규업무:공급방법/장소,20030214,20180807,공장을 새로 짓는데 옆 공장에서 변압기용량이 남는다고 하여 일부를 연결하여 같이 사...,동일한 전기사용 장소내에서 전기를 사용하고 계약종별 또는 회계단위가 상이한 2이상의...,전기사용:전기사용계약
4,143,"27,540",20100,전기사용:신규업무:공급방법/장소,20030214,20180807,전기설비중 냉·난방기기가 있는데 계약전력은 어떻게 산정합니까.,여름철에만 사용하는 설비와 겨울철에만 사용하는 설비가 1Set로 구성되어 냉난방용 ...,전기사용:전기사용신청
...,...,...,...,...,...,...,...,...,...
346,1246,"1,989",20224,시스템:대외시스템:사이버지점,20071205,20170103,전기공급약관을 보고 싶습니다.,전기공급약관은 `전력거래약관(새창열림)`href=`http://cyber.kepco...,사이버지점:사이버지점 이용
347,1468,"4,054",20224,시스템:대외시스템:사이버지점,20130628,20170103,법인회원으로 가입하는 방법을 알려주세요.,"개인회원으로 가입하신 후, `기업서비스이용신청(새창열림)`href=`https://...",사이버지점:사이버지점 이용
348,1469,"2,880",20224,시스템:대외시스템:사이버지점,20130628,20170103,전기요금을 미리 계산해 볼 수 있나요.,`전기요금계산기(새창열림)`href=`http://cyber.kepco.co.kr/...,사이버지점:사이버지점 이용
349,1470,"6,323",20224,시스템:대외시스템:사이버지점,20130628,20170103,아파트 관리사무소입니다. 종합계약과 단일계약을 비교해보고 싶습니다.,`아파트·오피스텔요금비교(새창열림)`href=`http://cyber.kepco.c...,사이버지점:사이버지점 이용


In [46]:
faqs[['순번', '제목', '내용']]

,순번,제목,내용
0,554,1주택수가구 요금을 적용받고 있는데 가구수가 감소하면 어떻게 해야 합니까.,1주택수가구 요금을 적용받던 고객이 세입자 전출등 가구수가 감소하는 경우에는 1개월...
1,229,고객이 임시공급설비 설치공사비를 부담하여 한전에서 임시공급설비를 시설하였을 경우의 ...,임시공급설비를 한전에서 설치할 경우에는 상시전력 공급설비의 수급지점과 동일하게하여 ...
2,535,"변압기공동이용(모자거래)이란 무엇이며, 요금계산은 어떻게 합니까.",동일 전기사용장소내에 전기사용용도(계약종별)가 다르거나 회계단위가 상이한 2이상의 ...
3,117,공장을 새로 짓는데 옆 공장에서 변압기용량이 남는다고 하여 일부를 연결하여 같이 사...,동일한 전기사용 장소내에서 전기를 사용하고 계약종별 또는 회계단위가 상이한 2이상의...
4,143,전기설비중 냉·난방기기가 있는데 계약전력은 어떻게 산정합니까.,여름철에만 사용하는 설비와 겨울철에만 사용하는 설비가 1Set로 구성되어 냉난방용 ...
...,...,...,...
346,1246,전기공급약관을 보고 싶습니다.,전기공급약관은 `전력거래약관(새창열림)`href=`http://cyber.kepco...
347,1468,법인회원으로 가입하는 방법을 알려주세요.,"개인회원으로 가입하신 후, `기업서비스이용신청(새창열림)`href=`https://..."
348,1469,전기요금을 미리 계산해 볼 수 있나요.,`전기요금계산기(새창열림)`href=`http://cyber.kepco.co.kr/...
349,1470,아파트 관리사무소입니다. 종합계약과 단일계약을 비교해보고 싶습니다.,`아파트·오피스텔요금비교(새창열림)`href=`http://cyber.kepco.c...


In [47]:
!pip install konlpy

In [0]:
!pip install --upgrade h5py

     |████████████████████████████████| 2.9MB 2.6MB/s 
  Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0


In [0]:
#형태소 분석
import jpype
from konlpy.tag import Kkma

In [0]:
kkma = Kkma()
filter_kkma = ['NNG',  #보통명사
             'NNP',  #고유명사
             'OL' ,  #외국어
            ]

def tokenize_kkma(doc):
    jpype.attachThreadToJVM()
    token_doc = ['/'.join(word) for word in kkma.pos(doc)]
    return token_doc

def tokenize_kkma_noun(doc):
    jpype.attachThreadToJVM()
    token_doc = ['/'.join(word) for word in kkma.pos(doc) if word[1] in filter_kkma]
    return token_doc


In [0]:
#자바를 안하면 나오는 에러
#from konlpy.tag import Kkma
#kkma = Kkma()
#def tokenize_kkma(doc):
#    token_doc = ['/'.join(word) for word in kkma.pos(doc) ]
#    return token_doc

In [0]:
tokenize_kkma_noun(faqs['제목'][0])

['주택/NNG',
 '수가/NNG',
 '구/NNG',
 '요금/NNG',
 '적용/NNG',
 '받/VV',
 '있/VXV',
 '가구/NNG',
 '수가/NNG',
 '감소/NNG',
 '어떻/VA',
 '하/VV',
 '하/VV']

In [0]:
len(faqs)

351

In [0]:
# 리스트에서 각 문장부분 토큰화
token_faqs = []
for i in range(len(faqs)):
    token_faqs.append([tokenize_kkma_noun(faqs['제목'][i]), i ])

# Doc2Vec에서 사용하는 태그문서형으로 변경
tagged_faqs = [TaggedDocument(d, [int(c)]) for d, c in token_faqs]

In [0]:
tagged_faqs

In [0]:
    # make model
    import multiprocessing
    cores = multiprocessing.cpu_count()
    d2v_faqs = doc2vec.Doc2Vec(
                                    vector_size=100,
                                    alpha=0.025,
                                    min_alpha=0.025,
                                    hs=1,
                                    negative=0,
                                    dm=0,
                                    window=3,
                                    dbow_words = 1,
                                    min_count = 1,
                                    workers = cores,
                                    seed=0,
                                    epochs=100
                                    )
    d2v_faqs.build_vocab(tagged_faqs)

    # train document vectors
    for epoch in range(50):
        d2v_faqs.train(tagged_faqs,
                                   total_examples = d2v_faqs.corpus_count,
                                   epochs = d2v_faqs.epochs
                                   )
        d2v_faqs.alpha -= 0.0025 # decrease the learning rate
        d2v_faqs.min_alpha = d2v_faqs.alpha # fix the learning rate, no decay

In [14]:
d2v_faqs.vocabulary.sample

0.001

In [78]:
test_string = "건물을 새로 지을 때 임시전력은 어떻게 신청하나요"
tokened_test_string = tokenize_kkma_noun(test_string)
tokened_test_string

['건물/NNG', '때/NNG', '임시/NNG', '전력/NNG', '신청/NNG']

In [80]:
topn = 5
test_vector = d2v_faqs.infer_vector(tokened_test_string)
result = d2v_faqs.docvecs.most_similar([test_vector], topn=topn)
print(result)

for i in range(topn):
    print("{}위. {}, {} {}".format(i+1, result[i][1], result[i][0],faqs['제목'][result[i][0]] ))

[(184, 0.387592077255249), (266, 0.3763398826122284), (20, 0.3675689101219177), (250, 0.3201121389865875), (348, 0.29826658964157104)]
1위. 0.387592077255249, 184 주택용전력 고압요금이란 무엇입니까.
2위. 0.3763398826122284, 266 전기요금 자동이체 신청 및 해지방법을 알려주세요.
3위. 0.3675689101219177, 20 전기사용에 있어 한전과의 수급지점이 있다는데 수급지점에 대하여 자세히 알고 싶습니다.
4위. 0.3201121389865875, 250 착오로 전기요금을 두 번 냈는데 어떻게 해야 환불 받을 수 있습니까 .
5위. 0.29826658964157104, 348 전기요금을 미리 계산해 볼 수 있나요.


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [81]:
# 성능 측정
raten = 1
found = 0
for i in range(len(faqs)):
    tstr = faqs['제목'][i]
    ttok = tokenize_kkma_noun(tstr)
    tvec = d2v_faqs.infer_vector(ttok)
    re = d2v_faqs.docvecs.most_similar([tvec], topn = raten)
    for j in range(raten):
        if i == re[j][0]: found = found + 1

print("정확도 = {} % ({}/{} )  ".format(found/len(faqs),found, len(faqs)))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


정확도 = 0.5527065527065527 % (194/351 )  


In [66]:
# 모델 저장

# 모델 1
#d2v_faqs.save(os.path.join('data','/content/drive/My Drive/data/d2v_faqs_size100_min1_batch50_epoch100_nounonly_dm0.model'))

#모델 2
d2v_faqs.save(os.path.join('data','/content/drive/My Drive/data/d2v_faqs_size50_min1_batch50_epoch50_nounonly_dm0.model'))

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [67]:
# 모델 load
d2v_faqs_1 = doc2vec.Doc2Vec.load(os.path.join('data','/content/drive/My Drive/data/d2v_faqs_size100_min1_batch50_epoch100_nounonly_dm0.model'))
d2v_faqs_2 = doc2vec.Doc2Vec.load(os.path.join('data','/content/drive/My Drive/data/d2v_faqs_size50_min1_batch50_epoch50_nounonly_dm0.model'))


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [72]:
test_string = "건물을 새로 지을 때 임시전력은 어떻게 신청하나요"
tokened_test_string = tokenize_kkma_noun(test_string)
tokened_test_string

topn = 5
# 모델 1 추측
test_vector1 = d2v_faqs_1.infer_vector(tokened_test_string)
result1 = d2v_faqs_1.docvecs.most_similar([test_vector1], topn=topn)

# 모델 2 추측
test_vector2 = d2v_faqs_2.infer_vector(tokened_test_string)
result2 = d2v_faqs_2.docvecs.most_similar([test_vector2], topn=topn)

for i in range(topn):
    print("모델 1 {}위. {}, {} {}".format(i+1, result1[i][1], result1[i][0],faqs['제목'][result1[i][0]] ))

print('')
for i in range(topn):
    print("모델 2 {}위. {}, {} {}".format(i+1, result2[i][1], result2[i][0],faqs['제목'][result2[i][0]] ))

모델 1 1위. 0.4082043766975403, 119 공장을 신축하는데 전기가 필요합니다. 공장건설용 전기는 임시전력으로 사용하여야 한다고 하는데 공장가동에 필요한 수전설비를 미리 갖추어 사용할 경우 산업용전력을 적용받을 수 없습니까.
모델 1 2위. 0.3838210999965668, 189 비주거용 주택용전력 전기요금은 어떻게 적용됩니까.
모델 1 3위. 0.3509012758731842, 292 전기를 신청한 지 한참 지났는데 송전이 되지 않아 알아보니 휴전때문이라는데 휴전을 앞당겨 전기를 빨리 공급해 줄 수는 없습니까.
모델 1 4위. 0.3252938985824585, 261 전기요금 자동이체시 혜택을 알고싶습니다.
모델 1 5위. 0.31150299310684204, 121 산업용전력의 적용 대상 및 전기요금은 어떻게 됩니까.

모델 2 1위. 0.5585153698921204, 61 전기 신규사용 신청시 고객이 원하는 장소까지 일반 전기사업자인 한전부담으로 전기설비를 가설하여야 할 것으로 생각하는데 왜 고객이 시설부담금을 부담하여야 합니까.
모델 2 2위. 0.3957214653491974, 51 건설공사장에 전기가 필요하여 임시전력을 사용하고 싶은데 임시공급설비를 한전에서 시설하여 줄 경우 고객부담시설부담금은 어떻게 산정합니까.
모델 2 3위. 0.38039711117744446, 44 임시전력을 사용하고 있는 빌딩공사장인데 계약전력을 3kW/10kW로 증설할 경우 추가로 보증금을 납부하여야 합니까.
모델 2 4위. 0.37541264295578003, 60 새로 지은 건물이 지중화공급지역입니다. 저압 30kW 정도의 전기가 필요한데 시설부담금은 어떻게 산정합니까.
모델 2 5위. 0.3639354705810547, 221 자동이체 고객도 카카오페이 청구서를 신청할 수 있나요.


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [75]:
# 성능 측정
raten = 1
found1 = 0
found2 = 0

for i in range(len(faqs)):
    tstr = faqs['제목'][i]
    ttok = tokenize_kkma_noun(tstr)
    tvec1 = d2v_faqs_1.infer_vector(ttok)
    re1 = d2v_faqs_1.docvecs.most_similar([tvec1], topn = raten)
    tvec2 = d2v_faqs_2.infer_vector(ttok)
    re2 = d2v_faqs_2.docvecs.most_similar([tvec2], topn = raten)
    for j in range(raten):
        if i == re1[j][0]: found1 = found1 + 1
        if i == re2[j][0]: found2 = found2 + 1

print("모델 1 정확도 = {} % ({}/{} )  ".format(found1/len(faqs),found1, len(faqs)))
print("모델 2 정확도 = {} % ({}/{} )  ".format(found2/len(faqs),found2, len(faqs)))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


모델 1 정확도 = 0.6524216524216524 % (229/351 )  
모델 2 정확도 = 0.5868945868945868 % (206/351 )  
